<a href="https://colab.research.google.com/github/moh53/Graduation-Project-FMS/blob/NN-models/40MB/concat_dataset_40MB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
data_X=np.load('drive/MyDrive/X.npy')
data_Y=np.load('drive/MyDrive/Y.npy')


In [ ]:
data_IDC_X = torch.from_numpy(data_X)
data_IDC_Y = torch.from_numpy(data_Y)
print(data_IDC_X.shape)
print(data_IDC_Y.shape)

torch.Size([5547, 50, 50, 3])
torch.Size([5547])


In [ ]:
imgs0 = data_IDC_X[data_Y==0] # (0 = no IDC, 1 = IDC)
imgs1 = data_IDC_X[data_Y==1]
print("THE SHAPE OF NEGATIVE SAMPLES", imgs0.shape)
#print(imgs0)
print("THE SHAPE OF POSITIVE SAMPLES", imgs1.shape)

THE SHAPE OF NEGATIVE SAMPLES torch.Size([2759, 50, 50, 3])
THE SHAPE OF POSITIVE SAMPLES torch.Size([2788, 50, 50, 3])


In [ ]:

concate_dataset = torch.utils.data.ConcatDataset([imgs0, imgs1])
print(type(data_IDC_X))
print(data_IDC_X.shape)
type(concate_dataset)

<class 'torch.Tensor'>
torch.Size([5547, 50, 50, 3])


torch.utils.data.dataset.ConcatDataset

In [ ]:

datasets = [imgs0,imgs1]
for i in range(3):
    datasets.append(torch.utils.data.TensorDataset(torch.arange(i*10, (i+1)*10)))

dataset = torch.utils.data.ConcatDataset([imgs0,imgs1])
loader = torch.utils.data.DataLoader(
    dataset,
    shuffle=False,
    num_workers=0,
    batch_size=2
)

#for data in loader:
  #  print(data)

In [ ]:
validation_split = .2
shuffle_dataset = True
random_seed= 1

In [ ]:
# Creating data indices for training and validation splits:
dataset_size = len(concate_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]


In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:

train_loader = torch.utils.data.DataLoader(concate_dataset, batch_size=32,
                                           sampler=train_sampler,)
validation_loader = torch.utils.data.DataLoader(concate_dataset, batch_size=32,
                                                sampler=valid_sampler)
print(len(train_loader))
print(len(validation_loader))

139
35


In [ ]:

import torch
from torch import nn
from torch import optim

In [ ]:
model = nn.Sequential(
    nn.Linear( 3 * 50 * 50 , 64),
    nn.ReLU(),
    nn.Linear(64 , 64),
    nn.ReLU(),
    nn.Linear(64 , 2)
)

In [ ]:
optimiser = optim.SGD(model.parameters(), lr=1e-2)

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
n_epochs = 15 
batch_size = 20 

for epoch in range(n_epochs):
    losses = list()
    # X is a torch Variable
    #permutation = torch.randperm(data_IDC_X.size()[0])

    for batch_size in train_loader:
        #optimizer.zero_grad()

        #indices = permutation[i:i+batch_size]
        batch_x, batch_y = data_IDC_X[indices], data_IDC_Y[indices]
        b = batch_x.size(0)
        batch_x = batch_x.view(b,-1)
        x = batch_x.view(batch_x.size(0), -1)
        #x =torchvision.transforms.functional.to_tensor(x)
        x = torch.div(x,255)
        l = model(x)
        j = loss(l , batch_y)
        model.zero_grad()
        j.backward()
        optimiser.step()
        losses.append(j.item())
    print(f'Epoch{epoch + 1}, train loss : {torch.tensor(losses).mean():.2f}')
       
    losses = list()
       
    
    for batch_size in validation_loader:
        #optimizer.zero_grad()

        #indices = permutation[i:i+batch_size]
        batch_x, batch_y = data_IDC_X[indices], data_IDC_Y[indices]
        b = batch_x.size(0)
        batch_x = batch_x.view(b,-1)
        x = batch_x.view(batch_x.size(0), -1)
        #x =torchvision.transforms.functional.to_tensor(x)
        with torch.no_grad():
            x = torch.div(x,255)
        l = model(x)
        j = loss(l , batch_y)
        losses.append(j.item())
    print(f'Epoch{epoch + 1}, validation loss : {torch.tensor(losses).mean():.2f}')
    

Epoch1, train loss : 0.56
Epoch1, validation loss : 0.56
Epoch2, train loss : 0.55
Epoch2, validation loss : 0.55
Epoch3, train loss : 0.55
Epoch3, validation loss : 0.55
Epoch4, train loss : 0.55
Epoch4, validation loss : 0.54
Epoch5, train loss : 0.54
Epoch5, validation loss : 0.54
Epoch6, train loss : 0.54
Epoch6, validation loss : 0.53
Epoch7, train loss : 0.54
Epoch7, validation loss : 0.54
Epoch8, train loss : 0.53
Epoch8, validation loss : 0.53
Epoch9, train loss : 0.53
Epoch9, validation loss : 0.53
Epoch10, train loss : 0.53
Epoch10, validation loss : 0.52
Epoch11, train loss : 0.53
Epoch11, validation loss : 0.53
Epoch12, train loss : 0.52
Epoch12, validation loss : 0.52
Epoch13, train loss : 0.52
Epoch13, validation loss : 0.52
Epoch14, train loss : 0.52
Epoch14, validation loss : 0.51
Epoch15, train loss : 0.52
Epoch15, validation loss : 0.52
